In [1]:
import json
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from astropy.visualization import ImageNormalize, AsinhStretch
from astropy.wcs import WCS
from lsst.afw.image import ExposureF
from lsst.daf.butler import Butler
from reproject import reproject_interp
from tqdm.auto import tqdm

from single_object import create_wcs, get_cutout, load_object_and_forced, BAND_COLORS

repo = "/repo/main"
instrument = "LSSTComCam"
release = "w_2025_06"
collection = f"LSSTComCam/runs/DRP/DP1/{release}/DM-48810"
butler = Butler(repo, collections=collection)


hats_path = Path("/sdf/data/rubin/shared/lsdb_commissioning/hats/") / release

In [2]:
!rm animation/*.png

rm: cannot remove 'animation/*.png': No such file or directory


In [3]:
size = 200
crop_size = 100
bands_to_consider = 'grizy'

n_frames = 300

# oid = 2430439666873285985
# period = 0.23554

# oid = 2132990985234301955
# period = false

# oid = 2430518831710486438
# period = False

oid = 2132990985234302106
period = False

cutout_path = Path('cutouts')
cutout_path.mkdir(parents=True, exist_ok=True)
animation_path = Path('animation')
animation_path.mkdir(parents=True, exist_ok=True)

obj = load_object_and_forced(oid, hats_path)
if period:
    lc = obj.lc.assign(phase=obj.lc.midpointMjdTai % period / period)
else:
    lc = obj.lc


# vmax = 2e-4 * obj.lc.psfFlux.max()vmax = 2e-4 * obj.lc.psfFlux.max()
vmax = 1e-2 * obj.lc.psfFlux.max()


def make_fig(ph0, ph1, band1):
    fig, ax = plt.subplots(1, 2, figsize=(15, 5), gridspec_kw={'width_ratios': [2, 1]})

    fig.suptitle(f'OID = {oid}' + f' Period = {period:.5f} d' if period else '')

    if period:
        ax[0].set_xlabel('phase')
    else:
        ax[0].set_xlabel('MJD')
    ax[0].set_ylabel('mag')
    ax[0].invert_yaxis()
    if period:
        ax[0].set_xlim(0, 1)
    for b in bands_to_consider:
        i = lc.band == b
        if period:
            x_values = lc.phase[i]
        else:
            x_values = obj.lc.midpointMjdTai[i]
        ax[0].errorbar(x_values, lc.psfMag[i], lc.psfMagErr[i], fmt='o', color=BAND_COLORS[b], label=b, alpha=0.3)
    ax[0].vlines(ph0, *ax[0].get_ylim(), color='black', lw=4, label=f'phase = {ph0:.3f}' if period else f'MJD = {ph0:.1f}')
    ax[0].legend(loc='upper left')

    ax[1].set_frame_on(False)  # Hide the frame
    ax[1].set_xticks([])  # Remove x ticks
    ax[1].set_yticks([])  # Remove y ticks
    ax[1].set_aspect(1)
    if period:
        ax[1].set_title(f'phase = {ph1:.3f}, band = {band1}')
    else:
        ax[1].set_title(f'MJD = {ph1:.5f}, band = {band1}')

    return fig, ax


reproject_wcs = create_wcs(obj.coord_ra, obj.coord_dec, size=size)

if period:
    x_ = np.linspace(0, 1, n_frames)
else:
    x_ = np.linspace(lc.midpointMjdTai.min(), lc.midpointMjdTai.max(), n_frames)

for x in tqdm(x_):
    closest_index = (lc.query(f'band in {list(bands_to_consider)}')['phase' if period else 'midpointMjdTai'] - x).abs().idxmin()
    row = lc.loc[closest_index]
    
    if period:
        ph1 = row.phase
    else:
        ph1 = row.midpointMjdTai
    fig, ax = make_fig(x, ph1, row.band)

    fits_file = cutout_path / f'{row.forcedSourceId}.fits'
    json_file = cutout_path / f'{row.forcedSourceId}.json'
    if fits_file.exists() and json_file.exists():
        image = ExposureF.readFits(str(fits_file))
    else:
        data_id = dict(visit=row.visit, detector=row.detector, instrument="LSSTComCam")
        image, box = get_cutout(butler, data_id, obj.coord_ra, obj.coord_dec, size=size)
        image.writeFits(str(fits_file))
        cutout_box_min_corner = box.getCorners()[0]
        with open(json_file, 'w') as fh:
            json.dump({'corner0': {'x': cutout_box_min_corner.x, 'y': cutout_box_min_corner.y}}, fh)
    
    with open(json_file) as fh:
        json_data = json.load(fh)
    corner_x, corner_y = json_data['corner0']['x'], json_data['corner0']['y']

    wcs = WCS(image.wcs.getFitsMetadata())
    wcs.wcs.crpix[0] -= corner_x
    wcs.wcs.crpix[1] -= corner_y
    
    image_array, _footprint = reproject_interp(
        (image.getImage().getArray(), wcs),
        reproject_wcs,
        shape_out=(size, size),
        order='bicubic',
    )
    ident = (size - crop_size)//2
    crop_array = image_array[ident:size-ident, ident:size-ident]
    
    norm = ImageNormalize(crop_array, vmin=0, vmax=vmax, stretch=AsinhStretch(0.1))
    ax[1].imshow(crop_array, cmap='gray', origin='lower', norm=norm)

    fig.savefig(animation_path / f'{x:.5f}.png')
    plt.close()

/sdf/data/rubin/user/kostya/linccf/.venv/lib/python3.11/site-packages/lsdb/dask/join_catalog_data.py:332: RuntimeWarning: Right catalog does not have a margin cache. Results may be incomplete and/or inaccurate.
  warnings.warn(


  0%|          | 0/300 [00:00<?, ?it/s]

In [4]:
! ffmpeg -y -framerate 60 -pattern_type glob -i 'animation/*.png' -c:v libx264 -pix_fmt yuv420p animation.mp4

ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.3.0 (conda-forge gcc 12.3.0-10)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1718838163959/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1718838163959/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1718838163959/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1718838163959/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1718838163959/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-lib